In [1]:
import os  
import sys
import pathlib

repo_path = pathlib.Path('.').absolute().parent
src_path = repo_path / 'src'

sys.path.insert(0, str(src_path)) 

In [2]:
import pymongo
import fairscape_mds.models

## Potential Improvements

- [ ]  openLDAP memberOf Overlay


### Bitnami Container Customization

1. Added additional overlays, and moved them to the folder the container configures for overlays
    - container still needs to run through initalization before the overlay can be added

2. On container startup need the CONFIG ADMIN to preform two changes to existing documents
   - modify the modules loaded to include the memberOf
   - add a new document to each database that needs the overlay

3. On database init, add the following 

# LDAP Tests

- [ ] throughput benchmarking
- [ ] 

## OpenLDAP docker image config

### Environment Variables
- `LDAP_PORT_NUMBER: 1389`
- `LDAP_ROOT: 'dc=fairscape,dc=net'`
- `LDAP_USER_DC: users`
  - DC for the users organizational unit
- `LDAP_GROUP: fairscapeUsers`
  - Group used to group created users default readers
- `LDAP_ADMIN_USERNAME: admin`
- `LDAP_ADMIN_PASSWORD: adminpassword`
- `LDAP_SKIP_DEFAULT_TREE: no`
  - whether to skip creation of `LDAP_USERS`, `LDAP_PASSWORDS`, `LDAP_GROUP`
- `LDAP_ADD_SCHEMAS: yes`
  - Whether to add the extra schemas amoung OpenLDAPs distributed schemas
- `LDAP_EXTRA_SCHEMAS: 'cosine, inetorgperson, nis'`



## LDAP Schema

LDIF example for inetOrgPerson
```
version: 1
dn: cn=Barbara Jensen,ou=Product Development,dc=siroe,dc=com
objectClass: top
objectClass: person
objectClass: organizationalPerson
objectClass: inetOrgPerson
cn: Barbara Jensen
cn: Babs Jensen
displayName: Babs Jensen
sn: Jensen
givenName: Barbara
initials: BJJ
title: manager, product development
uid: bjensen
mail: bjensen@siroe.com
telephoneNumber: +1 408 555 1862
facsimileTelephoneNumber: +1 408 555 1992
mobile: +1 408 555 1941
roomNumber: 0209
carLicense: 6ABC246
o: Siroe
ou: Product Development
departmentNumber: 2604
employeeNumber: 42
employeeType: full time
preferredLanguage: fr, en-gb;q=0.8, en;q=0.7
labeledURI: http://www.siroe.com/users/bjensen My Home Page
```

### CRUD Operations

`pip install ldap3`

Search returns `ldap3.abstract.entry.Entry`

In [34]:
from typing import List, Dict

In [384]:
from ldap3 import Server, Connection, ALL, SUBTREE
from ldap3.core.exceptions import LDAPException, LDAPBindError
from ldap3.abstract.entry import Entry
import ldap3

ldapServer = 'ldap://localhost:1389'
ldapBaseDN = 'dc=fairscape,dc=net'
username = 'admin'
adminPassword = 'adminpassword'
adminRDN = f"cn=admin,{ldapBaseDN}"


In [385]:
def connectLDAPServer(
    ldapURI: str,
    userRDN,
    userPassword
) -> Connection:

    try:
        server = Server(ldapURI, get_info=ALL)
        
        # username and password can be configured during openldap setup
        connection = Connection(server,          
                                user=userRDN, 
                                password=userPassword)
        
        bind_response = connection.bind() # Returns True or False 
        return connection
    except LDAPBindError as e:
        connection = e

#### Config Setup for Overlay

In [479]:
configAdminConnection = connectLDAPServer(ldapServer, 'cn=configadmin,cn=config', 'configadminpassword')

# modify the config module list
moduleConfigDN = "cn=module{0},cn=config"
moduleConfigChanges = {
    "olcModuleLoad":  [
         (ldap3.MODIFY_ADD, "memberof.la"),
         (ldap3.MODIFY_ADD, "memberof.so"), 
        (ldap3.MODIFY_ADD, "refint.la"),
        (ldap3.MODIFY_ADD, "refint.so"),
   ],
}

addMemberOfModule = configAdminConnection.modify(dn=moduleConfigDN, changes=moduleConfigChanges)
print(f"Adding Member of Module Success: {addMemberOfModule}")



# add a new list of modules
#addModuleList = configAdminConnection.add(
#    dn="cn=module,cn=config",
#    attributes={
#        "objectClass": "olcModuleList",
#        "olcModuleLoad": "memberof.la",
#        "olcModuleLoad": "refint.la",
#        "olcModulePath": "/usr/lib/ldap"
#    }
#)

#print(addModuleList)

Adding Member of Module Success: False


In [480]:

refintOverlayDN="olcOverlay=refint,olcDatabase={2}mdb,cn=config"
refintOverlayAttributes= {
    "objectClass": "olcRefintConfig",

 #   "objectClass": "olcConfig",
 #   "objectClass": "top",
    "olcOverlay": "refint",
    "olcRefintAttribute": "memberof member manager owner"
}

print(configAdminConnection.add(dn=refintOverlayDN, object_class= "olcOverlayConfig",attributes=refintOverlayAttributes))

True


In [481]:

memberOfOverlayDN="olcOverlay=memberof,olcDatabase={2}mdb,cn=config"
memberOfOverlayAttributes={
    "objectClass": "olcMemberOf",
   "olcOverlay": "memberof",
 #  "olcMemberOfRefInt": "TRUE",
   "olcMemberOfGroupOC": "groupOfNames",
   "olcMemberOfMemberAD": "member",
   "olcMemberOfMemberOfAD": "memberOf"

}

print(configAdminConnection.add(dn=memberOfOverlayDN, attributes=memberOfOverlayAttributes))

True


True


True


In [497]:
# 
configAdminConnection.search(
    search_base="olcDatabase={2}mdb,cn=config",
    search_filter="(objectClass=*)",
    attributes=['*']
)
configAdminConnection.entries

[DN: olcDatabase={2}mdb,cn=config - STATUS: Read - READ TIME: 2024-07-05T21:22:36.427576
     objectClass: olcDatabaseConfig
                  olcMdbConfig
     olcDatabase: {2}mdb
     olcDbDirectory: /bitnami/openldap/data
     olcDbIndex: objectClass eq,pres
                 ou,cn,mail,surname,givenname eq,pres,sub
     olcDbMaxSize: 1073741824
     olcMonitoring: False
     olcRootDN: cn=admin,dc=fairscape,dc=net
     olcRootPW: b'{SSHA}N6CSNnmIwixSsRVDGeaIayPIWhW4yJPq'
     olcSuffix: dc=fairscape,dc=net,
 DN: olcOverlay={0}refint,olcDatabase={2}mdb,cn=config - STATUS: Read - READ TIME: 2024-07-05T21:22:36.427881
     objectClass: olcOverlayConfig
                  olcRefintConfig
     olcOverlay: {0}refint
     olcRefintAttribute: memberof member manager owner,
 DN: olcOverlay={1}memberof,olcDatabase={2}mdb,cn=config - STATUS: Read - READ TIME: 2024-07-05T21:22:36.428522
     objectClass: olcMemberOfConfig
                  olcConfig
     olcMemberOfGroupOC: groupOfNames
     olc

In [496]:
configAdminConnection.delete("olcOverlay={1}memberof,olcDatabase={2}mdb,cn=config")

True

In [483]:
# add the oldOverlay to the olcdDtabase={2}mdb adding the document
#
# dn: olcOverlay=memberof,olcDatabase={2}mdb,cn=config
# objectClass: olcOverlayConfig
# objectClass: olcMemberOf
# olcOverlay: memberof
# olcMemberOfRefint: TRUE

# dn: olcOverlay={1}refint,olcDatabase={2}mdb,cn=config
# objectClass: olcConfig
# objectClass: olcOverlayConfig
# objectClass: olcRefintConfig
# objectClass: top
# olcOverlay: {1}refint
# olcRefintAttribute: memberof member manager owner


#dn: olcOverlay=memberof,olcDatabase={2}hdb,cn=config
#objectClass: olcMemberOf
#objectClass: olcOverlayConfig
#objectClass: olcConfig
#objectClass: top
#olcOverlay: memberof
#olcMemberOfDangling: ignore
#olcMemberOfRefInt: TRUE
#olcMemberOfGroupOC: groupOfNames
#olcMemberOfMemberAD: member
#olcMemberOfMemberOfAD: memberOf


memberOfOverlayDN="olcOverlay=memberof,olcDatabase={2}mdb,cn=config"
memberOfOverlayAttributes={
     "objectClass": "olcMemberOf",
    "objectClass": "olcOverlayConfig",
     "objectClass": "olcConfig",
    "olcOverlay": "memberof",
    "olcMemberOfRefInt": "TRUE",
    "olcMemberOfGroupOC": "groupOfNames",
    "olcMemberOfMemberAD": "member",
    "olcMemberOfMemberOfAD": "memberOf"
}
    
print(configAdminConnection.add(dn=memberOfOverlayDN, object_class="olcMemberOf", attributes=memberOfOverlayAttributes))


True


In [440]:
help(configAdminConnection.add)

Help on method add in module ldap3.core.connection:

add(dn, object_class=None, attributes=None, controls=None) method of ldap3.core.connection.Connection instance
    Add dn to the DIT, object_class is None, a class name or a list
    of class names.

    Attributes is a dictionary in the form 'attr': 'val' or 'attr':
    ['val1', 'val2', ...] for multivalued attributes



In [344]:
# delete duplicate overlays
# configAdminConnection.delete("olcOverlay={0}memberof,olcDatabase={2}mdb,cn=config")

False

In [394]:
# search for overlay
ldapSearch(configAdminConnection,
           searchFilter='(objectClass=*)',
           searchBase='olcDatabase={2}mdb,cn=config',
           searchAttributes=['*']
          )

[DN: olcDatabase={2}mdb,cn=config - STATUS: Read - READ TIME: 2024-07-05T20:39:21.467282
     objectClass: olcDatabaseConfig
                  olcMdbConfig
     olcDatabase: {2}mdb
     olcDbDirectory: /bitnami/openldap/data
     olcDbIndex: objectClass eq,pres
                 ou,cn,mail,surname,givenname eq,pres,sub
     olcDbMaxSize: 1073741824
     olcMonitoring: False
     olcRootDN: cn=admin,dc=fairscape,dc=net
     olcRootPW: b'{SSHA}RYzOiVrhsN157M7axNQmUW3eIu2qPWYR'
     olcSuffix: dc=fairscape,dc=net,
 DN: olcOverlay={0}memberof,olcDatabase={2}mdb,cn=config - STATUS: Read - READ TIME: 2024-07-05T20:39:21.467382
     objectClass: olcOverlayConfig
     olcOverlay: {0}memberof]

In [427]:
results = ldapSearch(configAdminConnection,
           searchFilter='(objectClass=olcModuleList)',
           searchBase='cn=config',
           searchAttributes=['cn', 'olcmoduleload', 'olcmodulepath']
          )

results[0].entry_attributes_as_dict

{'olcModulePath': ['/opt/bitnami/openldap/libexec/openldap'],
 'cn': ['module{0}'],
 'olcModuleLoad': ['{0}pw-sha2.so', '{1}memberof.so', '{2}refint.so']}

## Adding memberOf Overlay

- backend.refint.ldif 
```
dn: cn=module,cn=config
cn: module
objectclass: olcModuleList
objectclass: top
olcmoduleload: refint.la
olcmodulepath: /opt/bitnami/openldap/libexec/openldap
 
dn: olcOverlay={1}refint,olcDatabase={2}mdb,cn=config
objectClass: olcConfig
objectClass: olcOverlayConfig
objectClass: olcRefintConfig
objectClass: top
olcOverlay: {1}refint
olcRefintAttribute: memberof member manager owner
```

- backend.memberof.ldif
```
dn: cn=module,cn=config
cn: module
objectClass: olcModuleList
objectClass: top
olcModulePath: /usr/lib/ldap
olcModuleLoad: memberof.la
 
dn: olcOverlay={0}memberof,olcDatabase={2}mdb,cn=config
objectClass: olcConfig
objectClass: olcMemberOf
objectClass: olcOverlayConfig
objectClass: top
olcOverlay: memberof
```

## Setup Basic Users


In [484]:
ldapAdminConnection = connectLDAPServer(ldapServer, adminRDN, adminPassword)

In [180]:
### Example of searching 


#ldapConnection.search(
#    search_base="dc=fairscape,dc=net", 
#    search_filter='(objectClass=*)'
#)

#found_entries = adminConnection.entries

#type(found_entries[0])

#entryJSON = found_entries[0].entry_to_json()
#found_entries[0].entry_attributes_as_dict

In [485]:
def ldapSearch(
    ldapConnection: Connection,
    searchFilter: str,
    searchBase: str = 'dc=fairscape,dc=net',
    searchAttributes: List[str] = ['cn', 'uid', 'o']
) -> List[Entry] :
    """ Function for searching ldap entries

    Arguments:
    ldapConnection: (ldap3.Connection)
    searchFilter: (str) ldap filter expressed as string
    searchBase: (str) the base of ldap tree to search
    searchAttributes: (List[str]) the attributes to return from the search

    Example:
    ```
    results = searchLDAP(
        ldapConn, 
        searchFilter='(objectClass=*)', 
        searchBase='dc=fairscape,dc=net', 
        searchAttributes=['cn', 'uid', 'o']
        )
    ```
    
    """
    try:
        ldapConnection.search(
            search_base=searchBase,
            search_filter=searchFilter,
            search_scope=SUBTREE,
            attributes=searchAttributes
        )
        return ldapConnection.entries
    except LDAPException as e:
        return e



# utility functions

def ldapGetUsers(
    ldapConnection: Connection
) -> List[Entry]:
    """ Function for returning all fairscape users from LDAP
    """
    return ldapSearch(
        ldapConnection, 
        searchFilter='(objectClass=inetOrgPerson)', 
        searchBase='ou=users,dc=fairscape,dc=net',
        searchAttributes=['cn', 'uid', 'o', "memberof"]
    )

def ldapGetGroups(
    ldapConnection: Connection
) -> List[Entry]:
    """ Utility function for returning all fairscape groups from LDAP
    """
    return ldapSearch(
        ldapConnection, 
        searchFilter='(objectClass=groupOfNames)', 
        searchBase='ou=groups,dc=fairscape,dc=net',
        searchAttributes=['cn', 'member', 'o', 'owner']
    )

def ldapAdd(
    ldapConnection: Connection, 
    distinguishedName: str,
    entryAttributes: Dict[str, str],
):
    """ Generic Function for adding entries to LDAP
    
    Arguments:
    ldapConnection: (ldap3.Connection) connection to ldap server
    distinguishedName: (str) formated distinguished name for entry
    entryAttributes: (Dict[str,str]) attributes to add to the entry
    """
    return ldapConnection.add(distinguishedName, attributes=entryAttributes)


# setup fairscape ldap
def ldapSetupOU(ldapConnection)-> bool:
    """
    """
    
    # check that default ou's dont exist
    ouResults = ldapSearch(ldapConnection,
           searchFilter='(objectClass=organizationalUnit)',
           searchBase='dc=fairscape,dc=net',
           searchAttributes=['ou']
          )
    
    def addUserOU():
        return ldapAdd(
            ldapConnection, 
            distinguishedName="ou=users,dc=fairscape,dc=net", 
            entryAttributes={"objectClass": "organizationalUnit", "ou": "users", "description": "organizational unit of fairscape users"}
        )

    def addGroupOU():
        return ldapAdd(
            ldapConnection, 
            distinguishedName="ou=groups,dc=fairscape,dc=net", 
            entryAttributes={"objectClass": "organizationalUnit", "ou": "groups", "description": "groups of fairscape users"}
        )

    def addOrgOU():
        '''
        add organizations to directory
        '''
        return ldapAdd(
            ldapConnection,
            distinguishedName="ou=orgs,dc=fairscape,dc=net",
            entryAttributes={"objectClass": "organizationalUnit", "ou": "orgs", "description": "organizations of fairscape users"}
        )
    
    #addUserOUResult = addUserOU()
    addGroupOUResult = addGroupOU()
    addOrgOUResult = addOrgOU()


    # TODO log
    #print(f"addUserOUResult: {addUserOUResult}")
    print(f"addGroupOUResult: {addGroupOUResult}")
    print(f"addOrgOUResult: {addOrgOUResult}")
    return None
    


In [486]:
ldapSetupOU(ldapAdminConnection)

addGroupOUResult: True
addOrgOUResult: True


In [271]:
# check that default ou's dont exist
ldapSearch(ldapConnection,
           searchFilter='(objectClass=organizationalUnit)',
           searchBase='dc=fairscape,dc=net',
           searchAttributes=['ou']
          )
    

ldap3.core.exceptions.LDAPSocketSendError('socket sending error[Errno 32] Broken pipe')

In [104]:
# add UVA as organization
ldapAdd(
    ldapConnection,
    distinguishedName="o=UVA,ou=orgs,dc=fairscape,dc=net",
    entryAttributes={"objectClass": "organization", "o": "UVA"}
)

# add nova as organization

True

In [498]:
# delete all users
userCN="mal8ch"
ldapAdminConnection.delete(f"cn={userCN},ou=users,{ldapBaseDN}")

LDAPSessionTerminatedByServerError: session terminated by server

In [487]:
# Add ALL example users

# function for adding a user with password and email
def addFairscapeUser(
    ldapConnection, 
    userCN: str, 
    userSN: str, 
    userGN: str, 
    userMail: str,
    userPassword: str
):

    return ldapAdd(
        ldapConnection,
        f"cn={userCN},ou=users,{ldapBaseDN}",
        entryAttributes={
            "objectClass": "inetOrgPerson",
            "objectClass": "Person",
            "cn": userCN,
            "sn": userSN,
            "gn": userGN,
            "mail": userMail,
            "userPassword": userPassword
        }
    )

addFairscapeUser(
    ldapAdminConnection,
    userCN="mal8ch",
    userSN="Levinson",
    userGN="Max",
    userMail="mal8ch@virginia.edu",
    userPassword="maxtest"
)

addFairscapeUser(
    ldapAdminConnection,
    userCN="twc8q",
    userSN="Clark",
    userGN="Tim",
    userMail="twc8q@virginia.edu",
    userPassword="timtest"
)

addFairscapeUser(
    ldapAdminConnection,
    userCN="ma3xy",
    userSN="Al Manir",
    userGN="Sadnan",
    userMail="ma3xy@virginia.edu",
    userPassword="sadnantest"
)

addFairscapeUser(
    ldapAdminConnection,
    userCN="justin",
    userSN="Niestroy",
    userGN="Justin",
    userMail="jniestroy@gmail.com",
    userPassword="justintest"
)

True

In [488]:
usersDN = "ou=users,dc=fairscape,dc=net"
adminsCN = [
    f"cn=justin,{usersDN}",
    f"cn=ma3xy,{usersDN}",
    f"cn=mal8ch,{usersDN}",
    f"cn=twc8q,{usersDN}"
]


# add admin user group
ldapAdd(
    ldapAdminConnection,
    distinguishedName="cn=admins,ou=groups,dc=fairscape,dc=net",
    entryAttributes={"objectClass": "groupOfNames", "cn": "admins", "member": adminsCN}
)

True

In [492]:
ldapAdminConnection.search(
    search_base="ou=users,dc=fairscape,dc=net",
    search_filter="(objectClass=*)",
    attributes=["*", "memberOf"]
)

True

In [493]:
results = ldapAdminConnection.entries

In [494]:
results

[DN: ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-07-05T21:20:52.423501
     objectClass: organizationalUnit
     ou: users,
 DN: cn=ma3xy,ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-07-05T21:20:52.423640
     cn: ma3xy
     givenName: Sadnan
     mail: ma3xy@virginia.edu
     objectClass: inetOrgPerson
     sn: Al Manir
     userPassword: b'sadnantest',
 DN: cn=twc8q,ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-07-05T21:20:52.423748
     cn: twc8q
     givenName: Tim
     mail: twc8q@virginia.edu
     objectClass: inetOrgPerson
     sn: Clark
     userPassword: b'timtest',
 DN: cn=admins,ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-07-05T21:20:52.423824
     cn: admins
     member: cn=fairscapeUser,ou=users,dc=fairscape,dc=net
     objectClass: groupOfNames,
 DN: cn=justin,ou=users,dc=fairscape,dc=net - STATUS: Read - READ TIME: 2024-07-05T21:20:52.423923
     cn: justin
     givenName: Justin
     mail: jniestroy@gm

In [438]:
ldapAdminConnection.modify(
    "cn=mal8ch,ou=users,dc=fairscape,dc=net",
    changes={"memberOf": (ldap3.MODIFY_ADD, ["cn=admins,ou=groups,dc=fairscape,dc=net"])}
)

#results = ldapGetUsers(ldapAdminConnection)

False

In [437]:
results[2].entry_attributes_as_dict

{'cn': ['twc8q'], 'memberof': [], 'o': [], 'uid': []}

In [165]:
# delete the admin user group
ldapConnection.delete(
    "cn=admins,ou=groups,dc=fairscape,dc=net"
)

True

In [ ]:
# get list of members
ldapAdminConnection.search(
    search_filter="(cn=admins)",
    search_base="ou=groups,dc=fairscape,dc=net",
    attributes=['member']
)

adminGroup = ldapAdminConnection.entries[0]

adminGroupAttributes = adminGroup.entry_attributes_as_dict

adminGroupMembers = adminGroupAttributes['member']

adminGroupMembers

In [371]:
# search the users
users = ldapGetUsers(ldapAdminConnection)

In [372]:
users[2].entry_attributes_as_dict

{'cn': ['twc8q'], 'memberof': [], 'o': [], 'uid': []}

In [302]:
# remove the specified user
adminGroupMembers.remove('cn=justin,ou=users,dc=fairscape,dc=net')

In [303]:
adminGroupMembers

['cn=ma3xy,ou=users,dc=fairscape,dc=net',
 'cn=mal8ch,ou=users,dc=fairscape,dc=net',
 'cn=twc8q,ou=users,dc=fairscape,dc=net']

In [305]:
ldapAdminConnection.modify(
    "cn=admins,ou=groups,dc=fairscape,dc=net",
    changes={"member": (ldap3.MODIFY_REPLACE, adminGroupMembers)},
)

LDAPSessionTerminatedByServerError: session terminated by server

In [160]:
def removeUserFromGroup(ldapConnection, userCN, groupCN):
    '''
    Remove a user from a Groups Members
    '''
    
    # find the current list of members
    groupResult = ldapConnection.search(
        search_filter=f"(cn={groupCN})",
        search_base="ou=groups,dc=fairscape,dc=net",
        attributes=["member"]
    )

    if not groupResult:
        raise Exception

    matches = ldapConnection.entries

    if len(matches) == 0:
        raise Exception

    elif len(matches) > 1:
        raise Exception

    # get the list of members from group attributes
    matchedGroup = ldapConnection.entries[0]
    matchedGroupAttributes = matchedGroup.entry_attributes_as_dict
    matchedGroupMembers = matchedGroupAttributes.get('member')

    userDN = f"cn={userCN},ou=users,{ldapBaseDN}"
    groupDN = f"cn={groupCN},ou=groups,{ldapBaseDN}"
    
    if userDN not in matchedGroupMembers:
        print(f"Warning: User ({userDN}) is not in members of group ({groupDN})")
        return True

    # remove the current user from the group
    matchedGroupMembers.remove(userDN)

    return ldapConnection.modify(
        groupDN,
        changes={"member": (ldap3.MODIFY_REPLACE, matchedGroupMembers)},
    )


In [161]:
def addUserToGroup(ldapConnection, userCN, groupCN):
    '''
    Add a user to a group's member list
    '''
    
    # find the current list of members
    groupResult = ldapConnection.search(
        search_filter=f"(cn={groupCN})",
        search_base="ou=groups,dc=fairscape,dc=net",
        attributes=["member"]
    )

    if not groupResult:
        raise Exception

    matches = ldapConnection.entries

    if len(matches) == 0:
        raise Exception

    elif len(matches) > 1:
        raise Exception

    # get the list of members from group attributes
    matchedGroup = ldapConnection.entries[0]
    matchedGroupAttributes = matchedGroup.entry_attributes_as_dict
    matchedGroupMembers = matchedGroupAttributes.get('member')

    
    userDN = f"cn={userCN},ou=users,{ldapBaseDN}"
    groupDN = f"cn={groupCN},ou=groups,{ldapBaseDN}"
    
    if userDN in matchedGroupMembers:
        print(f"Warning: User ({userDN}) is already in members of group ({groupDN})")
        return True

    # remove the current user from the group
    matchedGroupMembers.append(userDN)

    return ldapConnection.modify(
        groupDN,
        changes={"member": (ldap3.MODIFY_REPLACE, matchedGroupMembers)},
    )


In [52]:
preemo_uva_cn = "preemo-uva"
add_group_preemo_uva = ldapAdd(
    ldapConnection,
    distinguishedName= f"cn={preemo_uva_cn},ou=groups,dc=fairscape,dc=net",
    entryAttributes={"objectClass": "groupOfNames", "cn": preemo_uva_cn, "member": ["cn=fairscapeUser,ou=users,dc=fairscape,dc=net"]}
)
add_group_preemo_uva

True

In [53]:
preemo_nova_cn = "preemo-nova"
add_group_preemo_nova = ldapAdd(
    ldapConnection,
    distinguishedName= f"cn={preemo_nova_cn},ou=groups,dc=fairscape,dc=net",
    entryAttributes={"objectClass": "groupOfNames", "cn": preemo_nova_cn, "member": ["cn=fairscapeUser,ou=users,dc=fairscape,dc=net"]}
)
add_group_preemo_nova

True

In [59]:
# add user
add_mal8ch = ldapAdd(
    ldapConnection,
    distinguishedName="cn=mal8ch,ou=users,dc=fairscape,dc=net",
    entryAttributes={
        "objectClass": "inetOrgPerson", 
        "cn": "mal8ch", 
        "givenName": "Max", 
        "sn": "Levinson",
        "mail": "mal8ch@virginia.edu",
        "userPassword": "testpass",
        "o": "o=UVA,ou=orgs,dc=fairscape,dc=net"
    }
)
add_mal8ch

True

In [71]:
# delete user
delete_mal8ch =ldapAdminConnection.delete(dn='o,dc=fairscape,dc=net')
delete_mal8ch

True

In [50]:
from pydantic import BaseModel, Field

class ldapGroup(BaseModel):
    name: str = Field()
    description: str | None
    cn: str
    objectClass: str = "groupOfNames"
    member: List[str] = Field()

In [ ]:
def ldapAddGroup(ldapConnection: Connection, groupInstance: LDAPGroup):
    """
    """
    pass

def ldapAddUser(ldapConnection: Connection):
    """
    """
    pass

def ldapAddUserToGroup(ldapConnection: Connection):
    """
    """
    pass


def ldapRemoveUserFromGroup(ldapConnection: Connection):
    """
    """
    pass

In [ ]:
""" Create a new group """

def add_ldap_group():

    # set all the group attributes
    ldap_attr = {}
    # object class for group should be mentioned.
    ldap_attr['objectClass'] = ['top', 'posixGroup']
    ldap_attr['gidNumber'] = '500'

    # Bind connection to LDAP server
    ldap_conn = connect_ldap_server()

    try:
        # this will add group1 to the base directory tree
        response = ldap_conn.add('cn=group1,dc=testldap,dc=com', 
                                  attributes=ldap_attr)
    except LDAPException as e:
        response = (" The error is ", e)
    ldap_conn.unbind()
    return response

In [ ]:
""" add method takes a user_dn, objectclass and attributes as    dictionary  """
def add_new_user_to_group():
    
    # sample attributes 
    ldap_attr = {}
    ldap_attr['cn'] = "test user"
    ldap_attr['sn'] = "AD"

    # Bind connection to LDAP server
    ldap_conn = connect_ldap_server()

    # this will create testuser inside group1
    user_dn = "cn=testuser,cn=group1,dc=testldap,dc=com"

    try:
        # object class for a user is inetOrgPerson
        response = ldap_conn.add(dn=user_dn,
                                 object_class='inetOrgPerson',
                                 attributes=ldap_attr)
    except LDAPException as e:
        response = e
return response

In [ ]:
""" The delete method is used to delete an entry from ldap
     The user/group dn is required to delete an entry"""

def delete_user():
    ldap_conn = connect_ldap_server()
    # Provide the dn of the user to be deleted 
    try:
                      
      response=ldap_conn.delete(dn='cn=testuser,cn=group1,
                          dc=testldap,dc=com')
    except LDAPException as e:
       response = e
    return response

In [ ]:


# perform the Modify operation
c.modify('cn=user1,ou=users,o=company',
         {'givenName': [(MODIFY_REPLACE, ['givenname-1-replaced'])],
          'sn': [(MODIFY_REPLACE, ['sn-replaced'])]})
print(c.result)


In [ ]:

def add_existing_user_to_ldapgroup():

    # all attributes are required to add existing users
    ldap_attr = {}
    ldap_attr['uid'] = b'tuser'
    ldap_attr['cn'] = b'Test User'
    ldap_attr['uidNumber'] = b'1001'
    ldap_attr['gidNumber'] = b'2001'
    ldap_attr['objectClass'] =  [b'top', b'inetOrgPerson',  
                                 b'posixAccount']
    ldap_attr['sn'] = b'User'
    ldap_attr['homeDirectory'] = b'/home/users/tuser'

    conn = ldap.initialize(server_uri, bytes_mode=False)
    conn.simple_bind_s("cn=admin,dc=testldap,dc=com", "12345")
    dn_new = "cn=Test user,cn=group1,dc=testldap,dc=com"
    ldif = modlist.addModlist(ldap_attr)
    try:
        response = conn.add_s(dn_new, ldif)
    except ldap.error as e:
        response = e
    finally:
        conn.unbind()
    return response

### Fairscape LDAP

In [1]:
import ldap

In [2]:
ldapServer = 'ldap://localhost:1389'
ldapBaseDN = 'dc=fairscape,dc=net'
username = 'admin'
password = 'adminpassword'

#def searchLDAP(username, password, ldapBaseDN, search):

try:
    connect = ldap.initialize(ldapServer)
    connect.simple_bind_s(f'cn={username},{ldapBaseDN}', password)
    results = connect.search_s('dc=fairscape,dc=net', ldap.SCOPE_SUBTREE) 
    connect.unbind_s()
except ldap.INVALID_CREDENTIALS:
    pass
except ldap.SERVER_DOWN:
    pass


In [3]:
help(connect.search_s)

Help on method search_s in module ldap.ldapobject:

search_s(base, scope, filterstr=None, attrlist=None, attrsonly=0) method of ldap.ldapobject.SimpleLDAPObject instance



In [4]:
results

[('dc=fairscape,dc=net',
  {'objectClass': [b'dcObject', b'organization'],
   'dc': [b'fairscape'],
   'o': [b'example']}),
 ('ou=users,dc=fairscape,dc=net',
  {'objectClass': [b'organizationalUnit'], 'ou': [b'users']}),
 ('cn=fairscapeUser,ou=users,dc=fairscape,dc=net',
  {'cn': [b'User1', b'fairscapeUser'],
   'sn': [b'Bar1'],
   'objectClass': [b'inetOrgPerson', b'posixAccount', b'shadowAccount'],
   'userPassword': [b'fairscapePass'],
   'uid': [b'fairscapeUser'],
   'uidNumber': [b'1000'],
   'gidNumber': [b'1000'],
   'homeDirectory': [b'/home/fairscapeUser']}),
 ('cn=readers,ou=users,dc=fairscape,dc=net',
  {'cn': [b'readers'],
   'objectClass': [b'groupOfNames'],
   'member': [b'cn=fairscapeUser,ou=users,dc=fairscape,dc=net']})]

In [5]:
# login as fairscape User
connect = ldap.initialize(ldapServer)
connect.simple_bind_s(f'cn=fairscapeUser,ou=users,{ldapBaseDN}', 'fairscapePass')
results = connect.search_s('dc=fairscape,dc=net', ldap.SCOPE_SUBTREE) 
connect.unbind_s()

In [6]:
results

[('dc=fairscape,dc=net',
  {'objectClass': [b'dcObject', b'organization'],
   'dc': [b'fairscape'],
   'o': [b'example']}),
 ('ou=users,dc=fairscape,dc=net',
  {'objectClass': [b'organizationalUnit'], 'ou': [b'users']}),
 ('cn=fairscapeUser,ou=users,dc=fairscape,dc=net',
  {'cn': [b'User1', b'fairscapeUser'],
   'sn': [b'Bar1'],
   'objectClass': [b'inetOrgPerson', b'posixAccount', b'shadowAccount'],
   'userPassword': [b'fairscapePass'],
   'uid': [b'fairscapeUser'],
   'uidNumber': [b'1000'],
   'gidNumber': [b'1000'],
   'homeDirectory': [b'/home/fairscapeUser']}),
 ('cn=readers,ou=users,dc=fairscape,dc=net',
  {'cn': [b'readers'],
   'objectClass': [b'groupOfNames'],
   'member': [b'cn=fairscapeUser,ou=users,dc=fairscape,dc=net']})]

In [7]:
help(connect.add_s)

Help on method add_s in module ldap.ldapobject:

add_s(dn, modlist) method of ldap.ldapobject.SimpleLDAPObject instance



In [8]:
# add a test user
userRDN = 'cn=Max,ou=users,dc=fairscape,dc=net'
modlist = ''


In [11]:
ldap.modlist.addModlist()

AttributeError: module 'ldap' has no attribute 'modlist'

In [ ]:
# create group for fairscape users
userGroup = 'CN=fairscapeUsers,CN=Users,DC=fairscape,DC=net'

In [10]:
# login as fairscape User
connect = ldap.initialize(ldapServer)
connect.simple_bind_s(f'cn=fairscapeUser,ou=users,{ldapBaseDN}', 'fairscapePass')
results = connect.add_s(userRDN, ) 
connect.unbind_s()

TypeError: ('List_to_LDAPMods(): expected list of tuples', None)

In [ ]:
def addUser():
    pass

def addOrganization():
    pass

def addUserToOrganization():
    pass


def removeUserFromOrganization():
    pass


def findUser():
    pass


def addROCrate():
    pass


def addDataset():
    pass


def addSoftware():
    pass


def addComputation():
    pass

In [ ]:

# perform the Modify operation
c.modify('cn=user1,ou=users,o=company',
         {'givenName': [(ldap.MODIFY_REPLACE, ['givenname-1-replaced'])],
          'sn': [(ldap.MODIFY_REPLACE, ['sn-replaced'])]})